In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import gc
import sys
import csv


In [4]:
test_pitch_df = pd.read_csv("../Data/test_pitch.csv")



In [5]:
new_row_1 = []
new_row_2 = []
new_row_3 = []
df = train_pitch_df[['表裏','プレイ前ホームチーム得点数', 'プレイ前アウェイチーム得点数']]
for i in df.values.tolist():
    if i[0] == '表':
        new_row_1.append(i[1])
        new_row_2.append(i[2])
        new_row_3.append(abs(i[1]-i[2]))
    else:
        new_row_1.append(i[2])
        new_row_2.append(i[1])
        new_row_3.append(abs(i[1]-i[2]))
train_pitch_df['自チーム得点数'] = new_row_1
train_pitch_df['相手チーム得点数'] = new_row_2
train_pitch_df['点差'] = new_row_3
df = train_pitch_df[['プレイ前ボール数', 'プレイ前ストライク数']]
np = df.values
new_row = ['1球目']
for i in range(1,len(df)):
    j = i - 1
    s_count = np[i][1] - np[j][1]
    b_count = np[i][0] - np[j][0]
    
    if s_count == 1:
        new_row.append('ストライク')
    elif b_count == 1:
        new_row.append('ボール')
    elif s_count == 0 and b_count == 0:
        new_row.append('ファウル')
    else:
        new_row.append('1球目')
train_pitch_df['1球前ステータス'] = new_row
train_pitch_df.to_csv("../Data/test_pitch_add_column1.csv", index=False)

NameError: name 'train_pitch_df' is not defined

In [3]:
test_pitch_df_list = pd.read_csv("../Data/test_pitch.csv", chunksize=100000)

test_pitcher_df = pd.read_csv("../Data/test_player.csv")
columns_pitcher = ['投手_年度', '投手_チームID', '投手_チーム名', '投手_選手ID', '投手_選手名', 
                   '投手_育成選手F', '投手_背番号', '投手_位置', '投手_投', '投手_打',
                   '投手_身長', '投手_体重', '投手_生年月日', '投手_出身高校ID', '投手_出身高校名', 
                   '投手_出身大学ID', '投手_出身大学名', '投手_社会人','投手_ドラフト年', '投手_ドラフト種別', 
                   '投手_ドラフト順位', '投手_年俸', '投手_出身国', '投手_出身地', '投手_血液型']
test_pitcher_df.columns = columns_pitcher

test_batter_df = pd.read_csv("../Data/test_player.csv")
columns_batter = ['打者_年度', '打者_チームID', '打者_チーム名', '打者_選手ID', '打者_選手名', '打者_育成選手F', 
                  '打者_背番号', '打者_位置', '打者_投', '打者_打','打者_身長', '打者_体重', '打者_生年月日', 
                  '打者_出身高校ID', '打者_出身高校名', '打者_出身大学ID', '打者_出身大学名', '打者_社会人',
                  '打者_ドラフト年', '打者_ドラフト種別', '打者_ドラフト順位', '打者_年俸', '打者_出身国', 
                  '打者_出身地', '打者_血液型']

test_batter_df.columns = columns_batter



In [ ]:
firstloop = True

for test_pitch_df in test_pitch_df_list:
    new_row_1 = []
    new_row_2 = []
    new_row_3 = []
    df_a = test_pitch_df[['表裏','プレイ前ホームチーム得点数', 'プレイ前アウェイチーム得点数']]
    print('test:',len(test_pitch_df))
    for i in df_a.values.tolist():
        if i[0] == '表':
            new_row_1.append(i[1])
            new_row_2.append(i[2])
            new_row_3.append(abs(i[1]-i[2]))
        else:
            new_row_1.append(i[2])
            new_row_2.append(i[1])
            new_row_3.append(abs(i[1]-i[2]))
    test_pitch_df['自チーム得点数'] = new_row_1
    test_pitch_df['相手チーム得点数'] = new_row_2
    test_pitch_df['点差'] = new_row_3
    df_b = test_pitch_df[['プレイ前ボール数', 'プレイ前ストライク数']]
    np = df_b.values
    new_row = ['1球目']
    
    for i in range(1,len(df_b)):
        j = i - 1
        s_count = np[i][1] - np[j][1]
        b_count = np[i][0] - np[j][0]

        if s_count == 1:
            new_row.append('ストライク')
        elif b_count == 1:
            new_row.append('ボール')
        elif s_count == 0 and b_count == 0:
            new_row.append('ファウル')
        else:
            new_row.append('1球目')

    test_pitch_df['1球前ステータス'] = new_row
    print('test2:',len(test_pitch_df))

    #年度間で投手IDの被りあり、年度とセットで一意に決まる
    df_c = pd.merge(test_pitch_df,test_pitcher_df,left_on=["投手ID","年度"], right_on=["投手_選手ID","投手_年度"], how="left")
    print('test3:',len(df_c))
    del test_pitch_df, df_a, df_b, new_row, new_row_1, new_row_2, new_row_3,



    df_d = pd.merge(df_c,test_batter_df,left_on=["打者ID","年度"], right_on=["打者_選手ID","打者_年度"], how="left")
    print('test4:',len(df_d))
    
    if firstloop:
        df_d.to_csv("../Data/test_pitch_add_column2.csv", index=False, mode = 'w')
        firstloop = False
    else:
        df_d.to_csv("../Data/test_pitch_add_column2.csv", index=False, header=False, mode = 'a')
        
    print("\n")

test: 100000
test2: 100000
test3: 100000
test4: 100000


test: 100000
test2: 100000
test3: 100000
test4: 100000


test: 100000
test2: 100000
test3: 100000
test4: 100000


In [ ]:
len

In [5]:
#メモリの確認
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print ("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))
        

|            Variable Name|    Memory|
 ------------------------------------ 
|                       In|       136|
|         LinearRegression|      1064|
|                      Out|       248|
|                  b_count|        32|
|           columns_batter|       272|
|          columns_pitcher|       272|
|                      csv|        88|
|                       df|        96|
|                     df_c|  50565405|
|                     df_d|  74381575|
|                     exit|        64|
|                        f|       224|
|                       gc|        88|
|              get_ipython|        72|
|                        i|        28|
|                        j|        28|
|                        l|        72|
|             linear_model|        88|
|                       np|       112|
|     parallel_coordinates|       144|
|                       pd|        88|
|                      plt|        88|
|                     quit|        64|
|                  s_coun

In [ ]:
train_pitch_df = pd.read_csv("../Data/train_pitch_add_column2.csv")

In [ ]:
train_pitch_df1 = pd.read_csv("../Data/train_pitch_add_column1.csv")